In [1]:
import sklearn
import time
import pandas as pd   
from matplotlib import pyplot
from numpy import mean
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB, CategoricalNB
import numpy as np



In [2]:

def missing_value_counts(dataframe):
    missing_counts = dataframe.isnull().sum()
    missing_counts_df = pd.DataFrame({'Column': missing_counts.index, 'Missing_Values_Count': missing_counts.values})
    return missing_counts_df

In [28]:
def custom_impute(df):
    # Separate columns into numerical and categorical
    numerical_columns = df.select_dtypes(include=['number']).columns
    categorical_columns = df.select_dtypes(exclude=['number']).columns

    # Step 1: Impute Numerical Columns with Mean
    num_imputer = KNNImputer(n_neighbors=100)
    df[numerical_columns] = num_imputer.fit_transform(df[numerical_columns])

    # Step 2: Impute Categorical Columns with Mode
    cat_imputer = SimpleImputer(strategy='most_frequent')
    df[categorical_columns] = cat_imputer.fit_transform(df[categorical_columns])

    return df

In [29]:
def min_max_scale_dataframe(df):
    numerical_columns = df.select_dtypes(include=['number']).columns
    scaler = RobustScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

    return df

In [106]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# def convert_numerical_to_categorical(df, num_bins=10):
#     numerical_cols = df.select_dtypes(include=['number']).columns
#     df_copy = df.copy()

#     # Exclude the "hospital_death" column from numerical_cols
#     numerical_cols = [col for col in numerical_cols if col != "hospital_death"]

#     for col in numerical_cols:
#         bin_labels = [f"{col}_bin_{i}" for i in range(num_bins)]
#         df_copy[col] = pd.cut(df_copy[col], bins=num_bins, labels=bin_labels)

#     categorical_transformer = OneHotEncoder(sparse=False, handle_unknown='ignore')
#     new_categorical_cols = df_copy.select_dtypes(exclude=['number']).columns

#     combined_categorical = pd.get_dummies(df_copy[new_categorical_cols], drop_first=True)
#     df_copy.drop(numerical_cols, axis=1, inplace=True)

#     df_copy = pd.concat([df_copy, combined_categorical], axis=1)

#     return df_copy

# import pandas as pd

def convert_numerical_to_categorical(df, num_bins=10):
    numerical_cols = df.select_dtypes(include=['number']).columns
    df_copy = df.copy()

    # Exclude the "hospital_death" column from numerical_cols
    numerical_cols = [col for col in numerical_cols if col != "hospital_death"]

    for col in numerical_cols:
        # Use pd.cut to create categorical columns based on specified bins
        df_copy[col] = pd.cut(df_copy[col], bins=num_bins, labels=False)  # labels=False for integer labels

    return df_copy


In [107]:
df = pd.read_csv('train.csv')
imputed_df = custom_impute(df)
imputed_df



,RecordID,hospital_id,icu_id,ethnicity,gender,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem,...,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_potassium_max,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,immunosuppression,solid_tumor_with_metastasis,hospital_death
0,1.0,126.0,1931.0,Caucasian,M,Floor,transfer,Med-Surg ICU,Metabolic,Metabolic,...,86.0,93.0,86.0,173.35,4.3332,0.0100,0.0000,0.0,0.0,0.0
1,2.0,112.0,1544.0,African American,M,Accident & Emergency,admit,Med-Surg ICU,Cardiovascular,Cardiovascular,...,95.0,95.0,95.0,333.00,3.7000,0.0821,0.0418,0.0,0.0,0.0
2,3.0,153.0,1517.0,Caucasian,M,Floor,admit,MICU,Respiratory,Respiratory,...,162.0,174.0,162.0,160.00,4.2000,0.3800,0.1500,0.0,0.0,0.0
3,4.0,109.0,1811.0,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Neurological,Neurologic,...,140.0,163.0,140.0,258.00,3.8000,0.1200,0.0600,0.0,0.0,0.0
4,5.0,287.0,1845.0,Caucasian,F,Accident & Emergency,admit,CSICU,Cardiovascular,Cardiovascular,...,119.0,119.0,119.0,110.00,3.9000,0.1500,0.0700,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996.0,110.0,1689.0,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Musculoskeletal/Skin,Undefined Diagnoses,...,98.0,98.0,98.0,98.00,3.9000,0.0200,0.0100,0.0,0.0,0.0
49996,49997.0,155.0,1719.0,Caucasian,F,Floor,admit,Med-Surg ICU,Neurological,Neurologic,...,145.0,145.0,145.0,145.00,3.8000,0.0200,0.0000,0.0,0.0,0.0
49997,49998.0,93.0,1912.0,Caucasian,F,Accident & Emergency,transfer,Med-Surg ICU,Sepsis,Cardiovascular,...,85.0,115.0,85.0,117.00,4.2090,0.0900,0.0500,1.0,0.0,0.0
49998,49999.0,230.0,1720.0,Caucasian,M,Operating Room / Recovery,admit,Cardiac ICU,Cardiovascular,Cardiovascular,...,137.0,137.0,137.0,119.00,4.0000,0.0000,0.0000,0.0,0.0,0.0


In [108]:
min_max_scale_dataframe(imputed_df)
imputed_df

,RecordID,hospital_id,icu_id,ethnicity,gender,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem,...,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_potassium_max,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,immunosuppression,solid_tumor_with_metastasis,hospital_death
0,-1.00000,-0.666667,1.241270,Caucasian,M,Floor,transfer,Med-Surg ICU,Metabolic,Metabolic,...,-0.805556,-1.000000,-0.805556,0.266875,0.203143,-0.454545,-0.500835,0.0,0.0,0.0
1,-0.99996,-0.789474,0.012698,African American,M,Accident & Emergency,admit,Med-Surg ICU,Cardiovascular,Cardiovascular,...,-0.555556,-0.945946,-0.555556,2.262500,-0.701429,0.200909,0.196995,0.0,0.0,0.0
2,-0.99992,-0.429825,-0.073016,Caucasian,M,Floor,admit,MICU,Respiratory,Respiratory,...,1.305556,1.189189,1.305556,0.100000,0.012857,2.909091,2.003339,0.0,0.0,0.0
3,-0.99988,-0.815789,0.860317,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Neurological,Neurologic,...,0.694444,0.891892,0.694444,1.325000,-0.558571,0.545455,0.500835,0.0,0.0,0.0
4,-0.99984,0.745614,0.968254,Caucasian,F,Accident & Emergency,admit,CSICU,Cardiovascular,Cardiovascular,...,0.111111,-0.297297,0.111111,-0.525000,-0.415714,0.818182,0.667780,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.99984,-0.807018,0.473016,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Musculoskeletal/Skin,Undefined Diagnoses,...,-0.472222,-0.864865,-0.472222,-0.675000,-0.415714,-0.363636,-0.333890,0.0,0.0,0.0
49996,0.99988,-0.412281,0.568254,Caucasian,F,Floor,admit,Med-Surg ICU,Neurological,Neurologic,...,0.833333,0.405405,0.833333,-0.087500,-0.558571,-0.363636,-0.500835,0.0,0.0,0.0
49997,0.99992,-0.956140,1.180952,Caucasian,F,Accident & Emergency,transfer,Med-Surg ICU,Sepsis,Cardiovascular,...,-0.833333,-0.405405,-0.833333,-0.437500,0.025714,0.272727,0.333890,1.0,0.0,0.0
49998,0.99996,0.245614,0.571429,Caucasian,M,Operating Room / Recovery,admit,Cardiac ICU,Cardiovascular,Cardiovascular,...,0.611111,0.189189,0.611111,-0.412500,-0.272857,-0.545455,-0.500835,0.0,0.0,0.0


In [109]:
missing_value_counts(imputed_df)

,Column,Missing_Values_Count
0,RecordID,0
1,hospital_id,0
2,icu_id,0
3,ethnicity,0
4,gender,0
5,icu_admit_source,0
6,icu_stay_type,0
7,icu_type,0
8,apache_3j_bodysystem,0
9,apache_2_bodysystem,0


In [111]:
cat_df = convert_numerical_to_categorical(imputed_df)

In [112]:
cat_df

,RecordID,hospital_id,icu_id,ethnicity,gender,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem,...,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_potassium_max,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,immunosuppression,solid_tumor_with_metastasis,hospital_death
0,0,1,9,Caucasian,M,Floor,transfer,Med-Surg ICU,Metabolic,Metabolic,...,2,1,2,1,3,5,5,0,0,0.0
1,0,0,5,African American,M,Accident & Emergency,admit,Med-Surg ICU,Cardiovascular,Cardiovascular,...,2,1,2,4,2,5,5,0,0,0.0
2,0,2,4,Caucasian,M,Floor,admit,MICU,Respiratory,Respiratory,...,7,6,7,1,3,6,5,0,0,0.0
3,0,0,8,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Neurological,Neurologic,...,6,5,6,3,2,5,5,0,0,0.0
4,0,9,8,Caucasian,F,Accident & Emergency,admit,CSICU,Cardiovascular,Cardiovascular,...,4,2,4,0,2,5,5,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,9,0,6,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Musculoskeletal/Skin,Undefined Diagnoses,...,3,1,3,0,2,5,5,0,0,0.0
49996,9,3,7,Caucasian,F,Floor,admit,Med-Surg ICU,Neurological,Neurologic,...,6,4,6,1,2,5,5,0,0,0.0
49997,9,0,9,Caucasian,F,Accident & Emergency,transfer,Med-Surg ICU,Sepsis,Cardiovascular,...,2,2,2,0,3,5,5,9,0,0.0
49998,9,6,7,Caucasian,M,Operating Room / Recovery,admit,Cardiac ICU,Cardiovascular,Cardiovascular,...,5,4,5,0,2,5,5,0,0,0.0


In [113]:
# df_onehot = pd.get_dummies(cat_df)
# df_onehot
df_onehot = cat_df.copy()

In [ ]:

df_onehot.columns
# pd.DataFrame(df_onehot.columns)
df_onehot.dtypes

In [117]:
import pandas as pd
exclude_cols = ["hospital_death"]
for col in df_onehot.columns:
    if col not in exclude_cols:
        df_onehot[col] = df_onehot[col].astype(str)


In [118]:
df_onehot.dtypes

RecordID                          object
hospital_id                       object
icu_id                            object
ethnicity                         object
gender                            object
icu_admit_source                  object
icu_stay_type                     object
icu_type                          object
apache_3j_bodysystem              object
apache_2_bodysystem               object
age                               object
elective_surgery                  object
pre_icu_los_days                  object
apache_2_diagnosis                object
apache_3j_diagnosis               object
apache_post_operative             object
gcs_eyes_apache                   object
gcs_motor_apache                  object
gcs_unable_apache                 object
gcs_verbal_apache                 object
heart_rate_apache                 object
intubated_apache                  object
resprate_apache                   object
temp_apache                       object
ventilated_apach

In [134]:
X = df_onehot.loc[:, df_onehot.columns != "hospital_death"]
y = df_onehot[["hospital_death"]]

X.shape

(50000, 57)

In [120]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=500,max_depth=5, min_samples_leaf=5, min_samples_split=3, n_jobs=3)

model.fit(X, y) 

feature_importances = model.feature_importances_

feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

selected_features = feature_importance_df['Feature'][:15]

# X = X[selected_features]

ValueError: could not convert string to float: 'Caucasian'

In [133]:
df_onehot

,RecordID,hospital_id,icu_id,ethnicity,gender,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem,...,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_potassium_max,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,immunosuppression,solid_tumor_with_metastasis,hospital_death
0,0,1,9,2,1,1,2,5,5,3,...,2,1,2,1,3,5,5,0,0,0.0
1,0,0,5,0,1,0,0,5,0,0,...,2,1,2,4,2,5,5,0,0,0.0
2,0,2,4,2,1,1,0,4,8,6,...,7,6,7,1,3,6,5,0,0,0.0
3,0,0,8,2,1,0,0,5,7,4,...,6,5,6,3,2,5,5,0,0,0.0
4,0,9,8,2,0,0,0,1,0,0,...,4,2,4,0,2,5,5,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,9,0,6,2,1,0,0,5,6,8,...,3,1,3,0,2,5,5,0,0,0.0
49996,9,3,7,2,0,1,0,5,7,4,...,6,4,6,1,2,5,5,0,0,0.0
49997,9,0,9,2,0,0,2,5,9,0,...,2,2,2,0,3,5,5,9,0,0.0
49998,9,6,7,2,1,2,0,3,0,0,...,5,4,5,0,2,5,5,0,0,0.0


In [128]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# List of columns to label encode
categorical_columns = ["ethnicity", "gender", "icu_admit_source", "icu_stay_type", "icu_type", "apache_3j_bodysystem", "apache_2_bodysystem"]

# Apply label encoding to each column
for col in categorical_columns:
    df_onehot[col] = label_encoder.fit_transform(df_onehot[col])


In [135]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [122]:
selected_features

465    apache_4a_icu_death_prob_apache_4a_icu_death_p...
456    apache_4a_hospital_death_prob_apache_4a_hospit...
459    apache_4a_hospital_death_prob_apache_4a_hospit...
460    apache_4a_hospital_death_prob_apache_4a_hospit...
199            ventilated_apache_ventilated_apache_bin_9
                             ...                        
415                      h1_sysbp_min_h1_sysbp_min_bin_9
406                      h1_sysbp_max_h1_sysbp_max_bin_9
306    h1_diasbp_noninvasive_min_h1_diasbp_noninvasiv...
370                h1_resprate_max_h1_resprate_max_bin_9
216    d1_diasbp_noninvasive_min_d1_diasbp_noninvasiv...
Name: Feature, Length: 400, dtype: object

In [136]:
model = CategoricalNB()

In [137]:
X.shape

(50000, 57)

In [48]:

def roc_auc_cv(model, X, y):
    cv = RepeatedKFold(n_splits=10, n_repeats=2, random_state=1)
    predicted_probabilities = cross_val_score(model, X, y, scoring="roc_auc", cv=cv, n_jobs=-1)

    return mean(predicted_probabilities)


In [18]:
# def find_best_parameters(X, y):
    
#     param_grid = {
#         'var_smoothing': [1e-13, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 
#                           0.09, 0.095, 0.097, 0.099, 0.1, 0.101, 0.103, 0.105, 0.11]
#     }
    

#     nb = GaussianNB()

#     grid_search = GridSearchCV(estimator=nb, param_grid=param_grid, cv=10, scoring='roc_auc')

#     grid_search.fit(X, y)

#     best_params = grid_search.best_params_
#     best_score = grid_search.best_score_

#     return best_params, best_score

# find_best_parameters(X, y)


In [50]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import roc_auc_score
import numpy as np

def roc_auc_cv_cnb(model, X, y):
    cv = RepeatedKFold(n_splits=10, n_repeats=2, random_state=1)
    y_probabilities = cross_val_predict(model, X, y, cv=cv, method='predict_proba', n_jobs=-1)

    # Assuming binary classification, you can select the positive class
    # probability (usually class 1)
    y_scores = y_probabilities[:, 1]

    # Calculate ROC AUC score
    roc_auc = roc_auc_score(y, y_scores)

    return roc_auc


In [60]:
from sklearn.model_selection import cross_val_score, RepeatedKFold
from sklearn.metrics import roc_auc_score
import numpy as np

def mean_roc_auc_cv_cnb(model, X, y):
    cv = RepeatedKFold(n_splits=10, n_repeats=2, random_state=1)
    roc_auc_scores = []

    for train_idx, test_idx in cv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)
        y_probs = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_probs)
        roc_auc_scores.append(roc_auc)

    mean_roc_auc = np.mean(roc_auc_scores)
    return mean_roc_auc


In [138]:
X_train

,RecordID,hospital_id,icu_id,ethnicity,gender,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem,...,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_potassium_max,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,immunosuppression,solid_tumor_with_metastasis
22204,4,9,7,2,0,0,0,6,7,4,...,1,3,1,3,1,2,0,0,0,0
16459,3,2,3,2,1,0,0,5,7,4,...,3,4,3,4,1,1,5,5,0,0
30593,6,1,6,2,1,2,0,6,0,0,...,4,4,4,4,1,2,5,5,0,0
40241,8,2,3,2,1,0,0,5,0,0,...,2,2,2,2,1,7,5,5,0,0
11995,2,1,5,0,0,1,0,5,8,6,...,1,3,1,3,2,4,7,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16304,3,8,1,2,0,0,0,3,0,0,...,4,5,4,5,1,4,5,5,0,0
79,0,1,7,5,0,0,0,5,0,0,...,4,6,4,5,0,3,5,5,0,0
12119,2,6,3,3,0,2,0,5,1,1,...,4,1,4,1,1,2,5,5,0,0
14147,2,9,5,2,1,0,0,5,0,0,...,2,4,2,4,3,2,5,5,0,0


In [189]:
model = CategoricalNB(alpha=34)

model.fit(X_train, y_train)
md_probs = model.predict_proba(X_test)
md_probs = md_probs[:,1]
md_auc = roc_auc_score(y_test, md_probs)
md_auc

/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8489674765604592

In [190]:
model.fit(X, y)

/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CategoricalNB(alpha=34)

In [194]:
def generate_predictions_for_model(model, test_file, output_file):
    
    df_test = pd.read_csv(test_file)
    record_ids = df_test["RecordID"]
    
    df_test_imputed = custom_impute(df_test)
    cat_df = convert_numerical_to_categorical(df_test_imputed)
    # df_test_onehot = pd.get_dummies(cat_df)
    df_test_onehot = cat_df.copy()
    
    # df_test_onehot = pd.get_dummies(df_test_imputed)
    # min_max_scale_dataframe(df_test_onehot)
    
    X_test = df_test_onehot.loc[:, df_test_onehot.columns != "hospital_death"]
    # X_test = X_test[selected_features]
    # Generate predictions using the model
    probs = model.predict_proba(X_test)
    probs = probs[:, 1]
    
    # Create a DataFrame for the results
    result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
    # Save the results to a CSV file
    result.to_csv(output_file, index=False, header=["RecordID", "hospital_death"])
    
generate_predictions_for_model(model, "test.csv", "results49.csv")


ValueError: invalid literal for int() with base 10: 'Caucasian'